In [51]:
# !pip install pandas openpyxl
import pandas as pd
import numpy as np

### Intiial

In [49]:
try:
    excel_df = pd.read_excel(r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out.xlsx')
    print("Excel file loaded successfully.")
except FileNotFoundError:
    print("Error: 'your_excel_file.xlsx' not found. Please upload it to Colab or provide the correct path.")
try:
    csv_df = pd.read_csv(r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\ed_results05_90BW.csv')
    print("CSV file loaded successfully.")
except FileNotFoundError:
    print("Error: 'your_csv_file.csv' not found. Please upload it to Colab or provide the correct path.")

Excel file loaded successfully.
CSV file loaded successfully.


In [45]:
excel_df.head()
# print(excel_df.columns.to_list())

# csv_df.head()
# print(csv_df.columns.to_list())

,ImageName,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40
0,BATCH05001.jpg,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,B,...,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BATCH05002.jpg,D,A,D,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D
2,BATCH05003.jpg,C,B,D,A,B,C,B,C,D,...,A,B,C,D,B,C,D,A,B,C
3,BATCH05004.jpg,A,C,C,A,D,B,A,C,D,...,A,B,C,B,A,A,D,C,D,B
4,BATCH05005.jpg,C,A,B,B,D,A,B,D,C,...,C,C,B,NaN,C,D,A,B,A,C


In [46]:
columns_to_drop_csv = ['Roll Number', 'Registration Number', 'Question Booklet Number']
# Using errors='ignore' ensures the code doesn't break if a column is already missing
csv_df = csv_df.drop(columns=columns_to_drop_csv, errors='ignore')

if not excel_df.empty and not csv_df.empty:
    if 'Image Name' in csv_df.columns and 'ImageName' not in csv_df.columns:
        csv_df = csv_df.rename(columns={'Image Name': 'ImageName'})
        print("\nRenamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.")
    question_columns = [col for col in excel_df.columns if col.startswith('Q')]
    merged_df = pd.merge(excel_df, csv_df, on='ImageName', how='outer', suffixes=('_excel', '_csv'))

    if not merged_df.empty:
        found_differences = False
        for index, row in merged_df.iterrows():
            image_name = row['ImageName']
            
            # Check if ImageName exists in both original DFs (important if using outer merge)
            in_excel = image_name in excel_df['ImageName'].values
            in_csv = image_name in csv_df['ImageName'].values

            row_has_diff = False
            output_lines = []

            for q_col in question_columns:
                excel_value = row.get(f'{q_col}_excel')
                csv_value = row.get(f'{q_col}_csv')

                # Treat NaN values for comparison: None if NaN, otherwise the value
                excel_comp_value = None if pd.isna(excel_value) else excel_value
                csv_comp_value = None if pd.isna(csv_value) else csv_value

                if excel_comp_value != csv_comp_value:
                    row_has_diff = True
                    output_lines.append(f"    {q_col}\t{excel_value}\t{csv_value}")
            
            # Also check if the ImageName was only in one of the files
            if not in_excel and in_csv:
                row_has_diff = True
                output_lines.insert(0, f"    Note: '{image_name}' found in CSV but not Excel.")
            elif in_excel and not in_csv:
                row_has_diff = True
                output_lines.insert(0, f"    Note: '{image_name}' found in Excel but not CSV.")

            if row_has_diff:
                found_differences = True
                print(f"{image_name}:")
                for line in output_lines:
                    print(line)

        if not found_differences:
            print("No differences found between the Excel and CSV files for the common ImageNames and questions.")
    else:
        print("\nMerged DataFrame is empty. This might mean there's no common 'ImageName' or files are empty.")
else:
    print("\nCannot perform comparison as one or both dataframes are empty. Please check file paths and content.")


Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.
BATCH05010.jpg:
    Q16	B	A*B
BATCH05015.jpg:
    Q1	A	nan
    Q4	B*D	D
    Q8	A	nan
    Q10	A	nan
    Q12	B*D	B
    Q18	B	nan
    Q21	A	nan
    Q23	A	nan
    Q30	A	nan
    Q35	A	nan
    Q38	A	nan
BATCH05018.jpg:
    Q16	C	B*C
BATCH05020.jpg:
    Q13	C*D	D
    Q24	B*C*D	B*C
BATCH05021.jpg:
    Q1	C*D	C
BATCH05022.jpg:
    Q1	nan	A
    Q20	B	nan
    Q22	A*C	nan
    Q27	B	nan
    Q31	A*D	nan
    Q40	A	nan
BATCH05024.jpg:
    Q11	C	nan
BATCH05025.jpg:
    Q20	C	nan
BATCH05027.jpg:
    Q4	B	B*D
BATCH05028.jpg:
    Q15	A*B	A


In [50]:
columns_to_drop_csv = ['Roll Number', 'Registration Number', 'Question Booklet Number']
# Using errors='ignore' ensures the code doesn't break if a column is already missing
csv_df = csv_df.drop(columns=columns_to_drop_csv, errors='ignore')

if not excel_df.empty and not csv_df.empty:
    if 'Image Name' in csv_df.columns and 'ImageName' not in csv_df.columns:
        csv_df = csv_df.rename(columns={'Image Name': 'ImageName'})
        print("\nRenamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.")
    question_columns = [col for col in excel_df.columns if col.startswith('Q')]
    merged_df = pd.merge(excel_df, csv_df, on='ImageName', how='outer', suffixes=('_excel', '_csv'))

    if not merged_df.empty:
        found_differences = False
        for index, row in merged_df.iterrows():
            image_name = row['ImageName']
            
            # Check if ImageName exists in both original DFs (important if using outer merge)
            in_excel = image_name in excel_df['ImageName'].values
            in_csv = image_name in csv_df['ImageName'].values

            row_has_diff = False
            output_lines = []

            for q_col in question_columns:
                excel_value = row.get(f'{q_col}_excel')
                csv_value = row.get(f'{q_col}_csv')

                # Treat NaN values for comparison: None if NaN, otherwise the value
                excel_comp_value = None if pd.isna(excel_value) else excel_value
                csv_comp_value = None if pd.isna(csv_value) else csv_value

                if excel_comp_value != csv_comp_value:
                    row_has_diff = True
                    output_lines.append(f"    {q_col}\t{excel_value}\t{csv_value}")
            
            # Also check if the ImageName was only in one of the files
            if not in_excel and in_csv:
                row_has_diff = True
                output_lines.insert(0, f"    Note: '{image_name}' found in CSV but not Excel.")
            elif in_excel and not in_csv:
                row_has_diff = True
                output_lines.insert(0, f"    Note: '{image_name}' found in Excel but not CSV.")

            if row_has_diff:
                found_differences = True
                print(f"{image_name}:")
                for line in output_lines:
                    print(line)

        if not found_differences:
            print("No differences found between the Excel and CSV files for the common ImageNames and questions.")
    else:
        print("\nMerged DataFrame is empty. This might mean there's no common 'ImageName' or files are empty.")
else:
    print("\nCannot perform comparison as one or both dataframes are empty. Please check file paths and content.")


Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.
BATCH05013.jpg:
    Q11	A	A*B
BATCH05015.jpg:
    Q1	A	nan
    Q10	A	nan
    Q21	A	nan
    Q30	A	nan
BATCH05017.jpg:
    Q20	D	A*D
BATCH05018.jpg:
    Q16	C	B*C
BATCH05021.jpg:
    Q1	C*D	C
BATCH05022.jpg:
    Q1	nan	A
    Q16	nan	A
    Q19	A	nan
    Q22	A*C	nan
    Q27	B	nan
    Q31	A*D	D
    Q40	A	nan
BATCH05024.jpg:
    Q11	C	nan
    Q33	A	A*B
BATCH05028.jpg:
    Q15	A*B	A


### Structuring

In [52]:
# --- Configuration ---
EXCEL_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out.xlsx'  # Make sure to upload this to Colab
CSV_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\ed_results05_90BW.csv'      # Make sure to upload this to Colab
VERIFICATION_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\verification.csv' # Make sure to upload this to Colab
OUTPUT_EXCEL_FILE = 'comparison_results.xlsx'

In [53]:
# --- 1. Load DataFrames ---

print("--- Loading Data ---")

try:
    excel_df = pd.read_excel(EXCEL_FILE_PATH)
    print(f"Excel file '{EXCEL_FILE_PATH}' loaded successfully.")
    # print("Excel columns:", excel_df.columns.tolist()) # Uncomment for debugging
except FileNotFoundError:
    print(f"Error: '{EXCEL_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    excel_df = pd.DataFrame() # Empty DF to prevent errors

try:
    csv_df = pd.read_csv(CSV_FILE_PATH)
    print(f"CSV file '{CSV_FILE_PATH}' loaded successfully.")
    # print("CSV columns (before drop):", csv_df.columns.tolist()) # Uncomment for debugging
except FileNotFoundError:
    print(f"Error: '{CSV_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    csv_df = pd.DataFrame() # Empty DF to prevent errors

try:
    verification_df = pd.read_csv(VERIFICATION_FILE_PATH)
    print(f"Verification file '{VERIFICATION_FILE_PATH}' loaded successfully.")
    # print("Verification columns:", verification_df.columns.tolist()) # Uncomment for debugging
except FileNotFoundError:
    print(f"Error: '{VERIFICATION_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    verification_df = pd.DataFrame() # Empty DF to prevent errors

# --- 2. Drop Columns from CSV DataFrame ---

if not csv_df.empty:
    columns_to_drop_csv = ['Roll Number', 'Registration Number', 'Question Booklet Number']
    csv_df = csv_df.drop(columns=columns_to_drop_csv, errors='ignore')
    print("\nDropped specified columns from CSV DataFrame.")
    # print("CSV DataFrame columns after drop:", csv_df.columns.tolist()) # Uncomment for debugging
else:
    print("\nCSV DataFrame is empty, skipping column drop.")

# --- 3. Prepare Data for Comparison ---

comparison_results = [] # To store the final structured output
if not excel_df.empty and not csv_df.empty:
    # Standardize the ImageName column for merging
    if 'Image Name' in csv_df.columns and 'ImageName' not in csv_df.columns:
        csv_df = csv_df.rename(columns={'Image Name': 'ImageName'})
        print("Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.")
    if 'Image Name' in excel_df.columns and 'ImageName' not in excel_df.columns:
        excel_df = excel_df.rename(columns={'Image Name': 'ImageName'})
        print("Renamed 'Image Name' in Excel DataFrame to 'ImageName' for consistent merging.")

    # Get the list of question columns (Q1 to Q40) based on the excel_df's columns, excluding 'ImageName'
    question_columns = [col for col in excel_df.columns if col.startswith('Q')]
    if not question_columns:
        print("Warning: No 'Q' columns (Q1-Q40) found in Excel file. Cannot perform question-level comparison.")

    # Merge the dataframes on 'ImageName'
    merged_df = pd.merge(excel_df, csv_df, on='ImageName', how='outer', suffixes=('_excel', '_csv'))

    print("\n--- Performing Comparison and Collecting Differences ---")

    if not merged_df.empty and question_columns:
        # Prepare verification_df for easier lookup
        if not verification_df.empty:
            # Standardize ImageName in verification_df too
            if 'Image Name' in verification_df.columns and 'ImageName' not in verification_df.columns:
                verification_df = verification_df.rename(columns={'Image Name': 'ImageName'})

            # Set 'ImageName' as index for fast lookups
            verification_df_indexed = verification_df.set_index('ImageName')
            print("Verification DataFrame indexed for faster lookups.")
        else:
            verification_df_indexed = pd.DataFrame()
            print("Verification DataFrame is empty, cannot add option details.")


        for index, row in merged_df.iterrows():
            image_name = row['ImageName']
            
            # Check if ImageName exists in both original DFs (important if using outer merge)
            in_excel = image_name in excel_df['ImageName'].values if not excel_df.empty else False
            in_csv = image_name in csv_df['ImageName'].values if not csv_df.empty else False

            # Handle cases where ImageName is only in one file
            if not in_excel and in_csv:
                comparison_results.append({
                    'ImageName': image_name,
                    'QuestionNumber': 'N/A',
                    'Excel Result': 'Image not in Excel',
                    'CSV Result': 'Image in CSV',
                    'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan # Add options as NaN
                })
                continue # Move to the next ImageName

            elif in_excel and not in_csv:
                comparison_results.append({
                    'ImageName': image_name,
                    'QuestionNumber': 'N/A',
                    'Excel Result': 'Image in Excel',
                    'CSV Result': 'Image not in CSV',
                    'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan # Add options as NaN
                })
                continue # Move to the next ImageName

            for q_col in question_columns:
                excel_value = row.get(f'{q_col}_excel')
                csv_value = row.get(f'{q_col}_csv')

                # Using pd.isna for robust NaN checking
                is_excel_na = pd.isna(excel_value)
                is_csv_na = pd.isna(csv_value)

                # Compare values. If both are NaN, consider them equal (no difference to report)
                if not (is_excel_na and is_csv_na) and \
                   (is_excel_na != is_csv_na or (not is_excel_na and not is_csv_na and excel_value != csv_value)):
                    
                    result_row = {
                        'ImageName': image_name,
                        'QuestionNumber': q_col,
                        'Excel Result': excel_value,
                        'CSV Result': csv_value
                    }

                    # --- Fetching data from verification.csv ---
                    if not verification_df_indexed.empty and image_name in verification_df_indexed.index:
                        ver_row = verification_df_indexed.loc[image_name]
                        q_num_stripped = q_col.replace('Q', '') # e.g., '1' from 'Q1'

                        # Initialize option results
                        option_results = {'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan}

                        for option in ['A', 'B', 'C', 'D']:
                            # Column names in verification.csv are like '1A', 'Result 1A'
                            option_col_name = f"{q_num_stripped}{option}"
                            result_option_col_name = f"Result {q_num_stripped}{option}"
                            
                            # Prioritize 'Result XA' column if it exists and is not NaN
                            if result_option_col_name in ver_row and not pd.isna(ver_row[result_option_col_name]):
                                option_results[option] = ver_row[result_option_col_name]
                            # Otherwise, fall back to 'XA' column if it exists and is not NaN
                            elif option_col_name in ver_row and not pd.isna(ver_row[option_col_name]):
                                option_results[option] = ver_row[option_col_name]

                        result_row.update(option_results) # Add A, B, C, D values

                    else:
                        # If no verification data, add NaN for options
                        result_row.update({'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan})

                    comparison_results.append(result_row)
    else:
        print("Merged DataFrame is empty or no question columns found. No comparison can be made.")

else:
    print("\nCannot perform comparison as one or both dataframes (Excel/CSV) are empty. Please check file paths and content.")

# --- 4. Create and Save Output Excel ---

if comparison_results:
    output_df = pd.DataFrame(comparison_results)
    
    # Define the desired order of columns
    output_columns = ['ImageName', 'QuestionNumber', 'Excel Result', 'CSV Result', 'A', 'B', 'C', 'D']
    # Reindex the DataFrame to ensure desired column order, handling missing columns if any
    output_df = output_df.reindex(columns=output_columns, fill_value=np.nan)

    try:
        output_df.to_excel(OUTPUT_EXCEL_FILE, index=False)
        print(f"\n--- Comparison results saved to '{OUTPUT_EXCEL_FILE}' ---")
        print(f"Total differences/entries logged: {len(output_df)}")
    except Exception as e:
        print(f"\nError saving Excel file: {e}")
else:
    print("\nNo differences found or data to save. Output Excel file not created.")

--- Loading Data ---
Excel file 'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out.xlsx' loaded successfully.
CSV file 'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\ed_results05_90BW.csv' loaded successfully.
Verification file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\verification.csv' loaded successfully.

Dropped specified columns from CSV DataFrame.
Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.

--- Performing Comparison and Collecting Differences ---
Verification DataFrame indexed for faster lookups.

--- Comparison results saved to 'comparison_results.xlsx' ---
Total differences/entries logged: 18


### Visualisation

In [55]:
# !pip install pandas openpyxl pillow

In [ ]:
import pandas as pd
import numpy as np
import json
import os
from PIL import Image

from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage

# --- Configuration ---
EXCEL_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out - Copy.xlsx'  # Make sure to upload this to Colab
CSV_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\29072025\Output\BATCH9\options_BATCH9\ed_results_human.csv'      # Make sure to upload this to Colab
VERIFICATION_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\29072025\Output\BATCH9\options_BATCH9\verification.csv' # Make sure to upload this to Colab
JSON_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\29072025\Output\BATCH9\BATCH9.json' # Make sure to upload your JSON file here

ORIGINAL_IMAGES_FOLDER_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\29072025\Output\BATCH9\processed_BATCH9' # <--- CHANGE THIS PATH!

OUTPUT_EXCEL_FILE = 'comparison_results.xlsx'
CROPPED_IMAGES_DIR = 'cropped_question_images' # Folder to save cropped images


In [102]:
# import pandas as pd
# import numpy as np
# import json
# import os
# from PIL import Image

# from openpyxl import load_workbook
# from openpyxl.drawing.image import Image as XLImage

# # --- Configuration ---
# EXCEL_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out.xlsx'  # Make sure to upload this to Colab
# CSV_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\ed_results_human.csv'      # Make sure to upload this to Colab
# VERIFICATION_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\verification.csv' # Make sure to upload this to Colab
# JSON_FILE_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\BATCH05.json' # Make sure to upload your JSON file here

# ORIGINAL_IMAGES_FOLDER_PATH = r'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Input\BATCH05' # <--- CHANGE THIS PATH!

# OUTPUT_EXCEL_FILE = 'comparison_results.xlsx'
# CROPPED_IMAGES_DIR = 'cropped_question_images' # Folder to save cropped images


# --- 1. Load DataFrames and JSON ---

print("--- Loading Data ---")

# Load Excel
try:
    excel_df = pd.read_excel(EXCEL_FILE_PATH)
    print(f"Excel file '{EXCEL_FILE_PATH}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{EXCEL_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    excel_df = pd.DataFrame()

# Load CSV
try:
    csv_df = pd.read_csv(CSV_FILE_PATH)
    print(f"CSV file '{CSV_FILE_PATH}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{CSV_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    csv_df = pd.DataFrame()

# Load Verification CSV
try:
    verification_df = pd.read_csv(VERIFICATION_FILE_PATH)
    print(f"Verification file '{VERIFICATION_FILE_PATH}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{VERIFICATION_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    verification_df = pd.DataFrame()

# Load JSON Data and Parse Coordinates
json_image_coords = {} # To store extracted data: {image_name: {full_path: ..., questions: {Q_id: {coords}}}}
print("\n--- Loading and Parsing JSON Data ---")
try:
    with open(JSON_FILE_PATH, 'r') as f:
        json_content = json.load(f)
    print(f"JSON file '{JSON_FILE_PATH}' loaded successfully.")

    if "IMAGES" in json_content:
        for image_entry in json_content["IMAGES"]:
            full_image_path_from_json = image_entry.get("IMAGENAME", "")
            if full_image_path_from_json:
                # Extract just the filename (e.g., BATCH05001.jpg)
                image_name_only = os.path.basename(full_image_path_from_json)
                
                json_image_coords[image_name_only] = {
                    'full_image_path': full_image_path_from_json, # Store the full path from JSON
                    'questions': {}
                }

                for field_entry in image_entry.get("FIELDS", []):
                    field_name = field_entry.get("FIELD", "")
                    if field_name.startswith("question_"):
                        # Convert "question_1" to "Q1"
                        q_number_formatted = field_name.replace("question_", "Q")
                        
                        try:
                            # Convert coordinates to integers
                            x = int(field_entry.get("XCORD"))
                            y = int(field_entry.get("YCORD"))
                            w = int(field_entry.get("WIDTH"))
                            h = int(field_entry.get("HEIGHT"))
                            
                            json_image_coords[image_name_only]['questions'][q_number_formatted] = {
                                'x': x, 'y': y, 'width': w, 'height': h
                            }
                        except (ValueError, TypeError):
                            print(f"Warning: Could not parse coordinates for {image_name_only}, {field_name}. Skipping.")
                            continue
    else:
        print("Warning: 'IMAGES' key not found in JSON data.")

except FileNotFoundError:
    print(f"Error: '{JSON_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from '{JSON_FILE_PATH}'. Check file format.")
except Exception as e:
    print(f"An unexpected error occurred while loading JSON: {e}")

# Create directory for cropped images if it doesn't exist
if not os.path.exists(CROPPED_IMAGES_DIR):
    os.makedirs(CROPPED_IMAGES_DIR)
    print(f"Created directory: {CROPPED_IMAGES_DIR}")

# --- 2. Drop Columns from CSV DataFrame ---

if not csv_df.empty:
    columns_to_drop_csv = ['Roll Number', 'Registration Number', 'Question Booklet Number']
    csv_df = csv_df.drop(columns=columns_to_drop_csv, errors='ignore')
    print("\nDropped specified columns from CSV DataFrame.")
else:
    print("\nCSV DataFrame is empty, skipping column drop.")

# --- 3. Prepare Data for Comparison ---

comparison_results = [] # To store the final structured output
if not excel_df.empty and not csv_df.empty:
    # Standardize the ImageName column for merging
    if 'Image Name' in csv_df.columns and 'ImageName' not in csv_df.columns:
        csv_df = csv_df.rename(columns={'Image Name': 'ImageName'})
        print("Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.")
    # Assuming Excel already has 'ImageName' based on previous context, but good to check.
    if 'Image Name' in excel_df.columns and 'ImageName' not in excel_df.columns:
        excel_df = excel_df.rename(columns={'Image Name': 'ImageName'})
        print("Renamed 'Image Name' in Excel DataFrame to 'ImageName' for consistent merging.")

    # Get the list of question columns (Q1 to Q40) based on the excel_df's columns, excluding 'ImageName'
    question_columns = [col for col in excel_df.columns if col.startswith('Q')]
    if not question_columns:
        print("Warning: No 'Q' columns (Q1-Q40) found in Excel file. Cannot perform question-level comparison.")

    # Merge the dataframes on 'ImageName'
    merged_df = pd.merge(excel_df, csv_df, on='ImageName', how='outer', suffixes=('_excel', '_csv'))

    print("\n--- Performing Comparison and Collecting Differences ---")
    print("Differences will be logged. Cropped images for differing questions will be saved.")

    if not merged_df.empty and question_columns:
        # Prepare verification_df for easier lookup
        verification_df_indexed = pd.DataFrame() # Initialize as empty
        if not verification_df.empty:
            if 'Image Name' in verification_df.columns and 'ImageName' not in verification_df.columns:
                verification_df = verification_df.rename(columns={'Image Name': 'ImageName'})
            
            if 'ImageName' in verification_df.columns:
                verification_df_indexed = verification_df.set_index('ImageName')
                print("Verification DataFrame indexed for faster lookups.")
            else:
                print("Warning: 'ImageName' column not found in verification.csv after potential rename. Cannot use verification data.")
        else:
            print("Verification DataFrame is empty, cannot add option details.")

        for index, row in merged_df.iterrows():
            image_name = row['ImageName']
            
            # Check if ImageName exists in both original DFs
            in_excel = image_name in excel_df['ImageName'].values if not excel_df.empty else False
            in_csv = image_name in csv_df['ImageName'].values if not csv_df.empty else False

            # Handle cases where ImageName is only in one file
            if not in_excel and in_csv:
                # Log this difference with N/A for question number and appropriate notes
                comparison_results.append({
                    'ImageName': image_name,
                    'QuestionNumber': 'N/A',
                    'Excel Result': 'Image not in Excel',
                    'CSV Result': 'Image in CSV',
                    'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan, # Add options as NaN
                    'Cropped Image Link': np.nan # No specific question to crop
                })
                continue # Move to the next ImageName

            elif in_excel and not in_csv:
                comparison_results.append({
                    'ImageName': image_name,
                    'QuestionNumber': 'N/A',
                    'Excel Result': 'Image in Excel',
                    'CSV Result': 'Image not in CSV',
                    'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan, # Add options as NaN
                    'Cropped Image Link': np.nan # No specific question to crop
                })
                continue # Move to the next ImageName

            # Process questions for ImageNames present in both (or only one with specific question diffs)
            for q_col in question_columns:
                excel_value = row.get(f'{q_col}_excel')
                csv_value = row.get(f'{q_col}_csv')

                is_excel_na = pd.isna(excel_value)
                is_csv_na = pd.isna(csv_value)

                # Only log if values are different (including one being NaN and other not)
                if not (is_excel_na and is_csv_na) and \
                   (is_excel_na != is_csv_na or (not is_excel_na and not is_csv_na and excel_value != csv_value)):
                    
                    result_row = {
                        'ImageName': image_name,
                        'QuestionNumber': q_col,
                        'Excel Result': excel_value,
                        'CSV Result': csv_value
                    }

                    # --- Fetching data from verification.csv ---
                    option_results = {'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan}
                    if not verification_df_indexed.empty and image_name in verification_df_indexed.index:
                        ver_row = verification_df_indexed.loc[image_name]
                        q_num_stripped = q_col.replace('Q', '') # e.g., '1' from 'Q1'

                        for option in ['A', 'B', 'C', 'D']:
                            option_col_name = f"{q_num_stripped}{option}"
                            result_option_col_name = f"Result {q_num_stripped}{option}"
                            
                            if result_option_col_name in ver_row and not pd.isna(ver_row[result_option_col_name]):
                                option_results[option] = ver_row[result_option_col_name]
                            elif option_col_name in ver_row and not pd.isna(ver_row[option_col_name]):
                                option_results[option] = ver_row[option_col_name]
                    result_row.update(option_results)

                    # --- New: Fetching and Cropping Image ---
                    cropped_image_path_for_excel = np.nan # Default to NaN

                    if image_name in json_image_coords and q_col in json_image_coords[image_name]['questions']:
                        img_details = json_image_coords[image_name]
                        q_coords = img_details['questions'][q_col]
                        
                        # Construct the full path to the original image file
                        image_filename_to_open = os.path.join(ORIGINAL_IMAGES_FOLDER_PATH, os.path.basename(img_details['full_image_path']))
                        
                        try:
                            # Open the image using Pillow
                            with Image.open(image_filename_to_open) as img:
                                # Define bounding box (left, upper, right, lower)
                                box = (q_coords['x'], q_coords['y'],
                                       q_coords['x'] + q_coords['width'],
                                       q_coords['y'] + q_coords['height'])
                                
                                cropped_img = img.crop(box)
                                
                                # Define path to save the cropped image
                                cropped_filename = f"{image_name}_{q_col}.png"
                                cropped_save_path = os.path.join(CROPPED_IMAGES_DIR, cropped_filename)
                                
                                cropped_img.save(cropped_save_path)
                                print(f"Cropped and saved: {cropped_save_path}") # Uncomment for verbose output
                                
                                # Store the relative path for Excel to create a hyperlink
                                # For local viewing after downloading, a relative path is often best.
                                # The 'file://' prefix often helps Excel recognize it as a link.
                                cropped_image_path_for_excel = f"file://{os.path.abspath(cropped_save_path)}"


                        except FileNotFoundError:
                            print(f"Warning: Original image '{image_filename_to_open}' not found for cropping '{image_name} {q_col}'. Please ensure it is in '{ORIGINAL_IMAGES_FOLDER_PATH}' folder.")
                        except Exception as e:
                            print(f"Error cropping image '{image_name} {q_col}': {e}")
                    
                    result_row['Cropped Image Link'] = cropped_image_path_for_excel
                    comparison_results.append(result_row)
    else:
        print("Merged DataFrame is empty or no question columns found. No comparison can be made.")

else:
    print("\nCannot perform comparison as one or more required dataframes (Excel/CSV/JSON) are empty. Please check file paths and content.")

# --- 4. Create and Save Output Excel ---

if comparison_results:
    output_df = pd.DataFrame(comparison_results)
    
    # Define the desired order of columns
    output_columns = ['ImageName', 'QuestionNumber', 'Excel Result', 'CSV Result', 'A', 'B', 'C', 'D', 'Cropped Image Link']
    # Reindex the DataFrame to ensure desired column order, filling missing columns with NaN
    output_df = output_df.reindex(columns=output_columns, fill_value=np.nan)

    try:
        output_df.to_excel(OUTPUT_EXCEL_FILE, index=False)
        from openpyxl import load_workbook
        from openpyxl.drawing.image import Image as XLImage

        # Load the Excel file we just created
        wb = load_workbook(OUTPUT_EXCEL_FILE)
        ws = wb.active

        # Find column index of "Cropped Image Link"
        image_col_idx = None
        for idx, col_name in enumerate(ws[1], start=1):
            if col_name.value == "Cropped Image Link":
                image_col_idx = idx
                break

        # Set column width for images
        if image_col_idx:
            col_letter = ws.cell(row=1, column=image_col_idx).column_letter
            ws.column_dimensions[col_letter].width = 25  # make column wide enough

            for row in range(2, ws.max_row + 1):
                img_path = ws.cell(row=row, column=image_col_idx).value
                if img_path and isinstance(img_path, str) and os.path.exists(img_path.replace("file://", "")):
                    try:
                        img_path_clean = img_path.replace("file://", "")
                        img = XLImage(img_path_clean)

                        # --- Resize image (keep aspect ratio) ---
                        base_width = 120
                        w_percent = base_width / float(img.width)
                        h_size = int(float(img.height) * w_percent)
                        img.width = base_width
                        img.height = h_size

                        # Set row height to match image
                        ws.row_dimensions[row].height = h_size * 0.75  # Excel height conversion factor

                        # Insert image in the cell
                        cell_address = f"{col_letter}{row}"
                        ws.add_image(img, cell_address)

                        # Remove text path
                        ws.cell(row=row, column=image_col_idx).value = None

                    except Exception as e:
                        print(f"Error inserting image for row {row}: {e}")

        wb.save(OUTPUT_EXCEL_FILE)
        print(f"Images embedded into '{OUTPUT_EXCEL_FILE}' with corrected aspect ratio.")

    except Exception as e:
        print(f"\nError saving Excel file: {e}")
else:
    print("\nNo differences found or data to save. Output Excel file not created.")

--- Loading Data ---
Excel file 'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out - Copy.xlsx' loaded successfully.
CSV file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\29072025\Output\BATCH9\options_BATCH9\ed_results_human.csv' loaded successfully.
Verification file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\29072025\Output\BATCH9\options_BATCH9\verification.csv' loaded successfully.

--- Loading and Parsing JSON Data ---
JSON file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\BATCH05.json' loaded successfully.

Dropped specified columns from CSV DataFrame.
Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.

--- Performing Comparison and Collecting Differences ---
Differences will be logged. Cropped images for differing questions will be saved.
Verification DataFrame indexed for faster lookups.
Images embedded into 'comparison_results.xlsx' with corrected aspect ratio.


In [92]:
import pandas as pd

# Load the Excel file
excel1_file = r"D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\comparison_results.xlsx"  # Replace with your file path
df = pd.read_excel(excel1_file)

# Drop specified columns
columns_to_drop = ["ImageName", "CroppedImageLink"]
df = df.drop(columns=columns_to_drop, errors='ignore')  # 'ignore' avoids error if column doesn't exist

# Save as CSV
csv1_file = "output.csv"  # Desired output filename
df.to_csv(csv1_file, index=False)

print(f"CSV saved as {csv1_file}")

CSV saved as output.csv
